In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import random
from datetime import datetime, timedelta
import pickle
from fuzzywuzzy import fuzz, process
from fractions import Fraction
import time
import collections
import concurrent.futures
from collections import defaultdict
import json
import requests
from bs4 import BeautifulSoup
import statistics
import math
from webdriver_manager.chrome import ChromeDriverManager

In [12]:
today = f"{datetime.now():%Y%m%d}"
today = '20221024'
game_data, ltp = fotmob_game_data(f'https://www.fotmob.com/?date={today}')
all_odds = get_all_odds_links(ltp)
days_links = connect_links(game_data, all_odds)

[WDM] - Downloading:  12%|█▏        | 760k/6.29M [00:00<00:00, 7.74MB/s]<ipython-input-2-37732944da27>:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
<ipython-input-3-2d71f0bb9be1>:42: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
[WDM] - Downloading: 100%|██████████| 6.29M/6.29M [00:16<00:00, 7.74MB/s]

In [13]:
days_links

{'2022-10-24 13:25:00': [['cremonese-vs-sampdoria',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919131/matchfacts/cremonese-vs-sampdoria',
   'https://www.oddschecker.com/football/italy/serie-a/cremonese-v-sampdoria/winner']],
 '2022-10-24 13:55:00': [['aalborg-bk-vs-viborg',
   'DENMARK - SUPERLIGAEN',
   'https://www.fotmob.com/match/3887425/matchfacts/aalborg-bk-vs-viborg',
   ''],
  ['djurg%C3%A5rden-vs-gif-sundsvall',
   'SWEDEN - ALLSVENSKAN',
   'https://www.fotmob.com/match/3805104/matchfacts/djurg%C3%A5rden-vs-gif-sundsvall',
   'https://www.oddschecker.com/football/sweden/allsvenskan/djurgarden-v-gif-sundsvall/winner'],
  ['ifk-v%C3%A4rnamo-vs-ifk-norrk%C3%B6ping',
   'SWEDEN - ALLSVENSKAN',
   'https://www.fotmob.com/match/3805108/matchfacts/ifk-v%C3%A4rnamo-vs-ifk-norrk%C3%B6ping',
   'https://www.oddschecker.com/football/sweden/allsvenskan/djurgarden-v-gif-sundsvall/winner'],
  ['mj%C3%A4llby-vs-varbergs-bois-fc',
   'SWEDEN - ALLSVENSKAN',
   'https://www.fotm

In [14]:
days_links = {'2022-10-24 13:25:00': [['cremonese-vs-sampdoria',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919131/matchfacts/cremonese-vs-sampdoria',
   'https://www.oddschecker.com/football/italy/serie-a/cremonese-v-sampdoria/winner']],
 '2022-10-24 13:55:00': [
  ['djurg%C3%A5rden-vs-gif-sundsvall',
   'SWEDEN - ALLSVENSKAN',
   'https://www.fotmob.com/match/3805104/matchfacts/djurg%C3%A5rden-vs-gif-sundsvall',
   'https://www.oddschecker.com/football/sweden/allsvenskan/djurgarden-v-gif-sundsvall/winner'],
  ['mj%C3%A4llby-vs-varbergs-bois-fc',
   'SWEDEN - ALLSVENSKAN',
   'https://www.fotmob.com/match/3805110/matchfacts/mj%C3%A4llby-vs-varbergs-bois-fc',
   'https://www.oddschecker.com/football/sweden/allsvenskan/mjallby-aif-v-varbergs-bois-fc/winner']],
 '2022-10-24 13:55:15': [['sirius-vs-kalmar-ff',
   'SWEDEN - ALLSVENSKAN',
   'https://www.fotmob.com/match/3805109/matchfacts/sirius-vs-kalmar-ff',
   'https://www.oddschecker.com/football/sweden/allsvenskan/sirius-v-kalmar-ff/winner'],
  ['giresunspor-vs-ankarag%C3%BCc%C3%BC',
   'TÜRKIYE - SUPER LIG',
   'https://www.fotmob.com/match/3932491/matchfacts/giresunspor-vs-ankarag%C3%BCc%C3%BC',
   'https://www.oddschecker.com/football/turkey/super-lig/giresunspor-v-ankaragucu/winner'],
  ['hatayspor-vs-be%C5%9Fikta%C5%9F',
   'TÜRKIYE - SUPER LIG',
   'https://www.fotmob.com/match/3932492/matchfacts/hatayspor-vs-be%C5%9Fikta%C5%9F',
   'https://www.oddschecker.com/football/turkey/super-lig/hatayspor-v-besiktas/winner']],
 '2022-10-24 14:05:00': [['ifk-g%C3%B6teborg-vs-aik',
   'SWEDEN - ALLSVENSKAN',
   'https://www.fotmob.com/match/3805107/matchfacts/ifk-g%C3%B6teborg-vs-aik',
   'https://www.oddschecker.com/football/sweden/allsvenskan/ifk-goteborg-v-aik/winner']],
 '2022-10-24 15:40:00': [['sassuolo-vs-hellas-verona',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919137/matchfacts/sassuolo-vs-hellas-verona',
   'https://www.oddschecker.com/football/italy/serie-a/sassuolo-v-verona/winner']],
 '2022-10-24 15:55:00': [['west-ham-united-vs-afc-bournemouth',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3901060/matchfacts/west-ham-united-vs-afc-bournemouth',
   'https://www.oddschecker.com/football/english/premier-league/west-ham-v-bournemouth/winner'],
  ['celta-vigo-vs-getafe',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3918038/matchfacts/celta-vigo-vs-getafe',
   'https://www.oddschecker.com/football/spain/la-liga-primera/celta-vigo-v-getafe/winner']],
 '2022-10-24 16:10:00': [['rio-ave-vs-portimonense',
   'PORTUGAL - LIGA PORTUGAL',
   'https://www.fotmob.com/match/3937471/matchfacts/rio-ave-vs-portimonense',
   'https://www.oddschecker.com/football/portugal/primeira-liga/rio-ave-v-portimonense/winner']]}

In [15]:
open_file = open(f'links/{today}_links.pkl', "wb")
pickle.dump(days_links, open_file)
open_file.close()

In [2]:
xg_fm = ['UNITED STATES - MAJOR LEAGUE SOCCER', 'ENGLAND - CHAMPIONSHIP',
 'ENGLAND - PREMIER LEAGUE', 'GERMANY - BUNDESLIGA', 'FRANCE - LIGUE 1', 'AUSTRIA - BUNDESLIGA', 'PORTUGAL - LIGA PORTUGAL',
 'BELGIUM - FIRST DIVISION A', 'SWITZERLAND - SUPER LEAGUE', 'SPAIN - LALIGA', 'DENMARK - SUPERLIGAEN', 'MEXICO - LIGA MX APERTURA',
 'NETHERLANDS - EREDIVISIE', 'SCOTLAND - PREMIERSHIP', 'ITALY - SERIE A', 'CHAMPIONS LEAGUE GRP. A', 'CHAMPIONS LEAGUE GRP. B', 'CHAMPIONS LEAGUE GRP. C',
 'CHAMPIONS LEAGUE GRP. D', 'CHAMPIONS LEAGUE GRP. E', 'CHAMPIONS LEAGUE GRP. F', 'CHAMPIONS LEAGUE GRP. G',
 'CHAMPIONS LEAGUE GRP. H', 'EUROPA LEAGUE GRP. A', 'EUROPA LEAGUE GRP. B', 'EUROPA LEAGUE GRP. C', 'EUROPA LEAGUE GRP. D',
 'EUROPA LEAGUE GRP. E', 'EUROPA LEAGUE GRP. F', 'EUROPA LEAGUE GRP. G', 'EUROPA LEAGUE GRP. H', 'EUROPA CONFERENCE LEAGUE GRP. A',
 'EUROPA CONFERENCE LEAGUE GRP. B', 'EUROPA CONFERENCE LEAGUE GRP. C', 'EUROPA CONFERENCE LEAGUE GRP. D', 'EUROPA CONFERENCE LEAGUE GRP. E',
         'EUROPA CONFERENCE LEAGUE GRP. F', 'EUROPA CONFERENCE LEAGUE GRP. G', 'EUROPA CONFERENCE LEAGUE GRP. H',
        'NORWAY - ELITESERIEN','TÜRKIYE - SUPER LIG', 'GERMANY - 2. BUNDESLIGA','SWEDEN - ALLSVENSKAN',
        'ENGLAND - LEAGUE ONE', 'ENGLAND - LEAGUE TWO','AUSTRALIA - A-LEAGUE MEN']

def fotmob_game_data(url):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(3)
    driver.get(url)
    divs = driver.find_elements(By.XPATH, '//div[@class="css-1f0alcw-Group e7pc1841"]')
    today = url[-8:]
    league = {}
    leagues_to_play = []
    try:
        for count, bucket in enumerate(divs):
            league_name = bucket.find_elements(By.XPATH, '//a[@class="css-vfisw6-GroupTitleLink e7pc1842"]')[count].text
            if league_name not in xg_fm:
                continue
            leagues_to_play.append(league_name)
            url = bucket.find_elements(By.TAG_NAME, 'a')
            urls = [u.get_attribute('href') for u in url]
            time = bucket.text
            time = [datetime.strptime(i + today, '%H:%M%Y%m%d') + timedelta(minutes=55) for i in time.split('\n') if ':' in i]
            urls = urls[1:]
            names = [u.split('/')[-1] for u in urls]
            league[league_name] = {n: [u,t] for u, t, n in zip(urls, time, names)}
    except:
        pass
    return league, leagues_to_play

In [3]:
xg_leagues_o = {'UNITED STATES - MAJOR LEAGUE SOCCER':'world/usa/mls','ENGLAND - CHAMPIONSHIP':'english/championship',
              'ENGLAND - PREMIER LEAGUE': 'english/premier-league', 'GERMANY - BUNDESLIGA': 'germany/bundesliga',
              'FRANCE - LIGUE 1': 'france/ligue-1','AUSTRIA - BUNDESLIGA': 'austria/bundesliga',
              'PORTUGAL - LIGA PORTUGAL':'portugal/primeira-liga', 'BELGIUM - FIRST DIVISION A':'belgium/jupiler-pro-league',
              'SWITZERLAND - SUPER LEAGUE':'switzerland/super-league','SPAIN - LALIGA':'spain/la-liga-primera',
             'DENMARK - SUPERLIGAEN':'denmark/superligaen', 'MEXICO - LIGA MX APERTURA':'world/mexico/liga-mx',
             'NETHERLANDS - EREDIVISIE':'netherlands/eredivisie', 'SCOTLAND - PREMIERSHIP': 'scottish/premiership',
             'ITALY - SERIE A':'italy/serie-a','CHAMPIONS LEAGUE GRP. A': 'champions-league',
                'CHAMPIONS LEAGUE GRP. B': 'champions-league',
                'CHAMPIONS LEAGUE GRP. C': 'champions-league',
                'CHAMPIONS LEAGUE GRP. D': 'champions-league',
                'CHAMPIONS LEAGUE GRP. E': 'champions-league',
                'CHAMPIONS LEAGUE GRP. F': 'champions-league',
                'CHAMPIONS LEAGUE GRP. G': 'champions-league',
                'CHAMPIONS LEAGUE GRP. H': 'champions-league',
                'EUROPA LEAGUE GRP. A': 'europa-league',
                'EUROPA LEAGUE GRP. B': 'europa-league',
                'EUROPA LEAGUE GRP. C': 'europa-league',
                'EUROPA LEAGUE GRP. D': 'europa-league',
                'EUROPA LEAGUE GRP. E': 'europa-league',
                'EUROPA LEAGUE GRP. F': 'europa-league',
                'EUROPA LEAGUE GRP. G': 'europa-league',
                'EUROPA LEAGUE GRP. H': 'europa-league',
             'EUROPA CONFERENCE LEAGUE GRP. A': 'uefa-europa-conference-league',
                'EUROPA CONFERENCE LEAGUE GRP. B': 'uefa-europa-conference-league',
                'EUROPA CONFERENCE LEAGUE GRP. C': 'uefa-europa-conference-league',
                'EUROPA CONFERENCE LEAGUE GRP. D': 'uefa-europa-conference-league',
                'EUROPA CONFERENCE LEAGUE GRP. E': 'uefa-europa-conference-league',
                'EUROPA CONFERENCE LEAGUE GRP. F': 'uefa-europa-conference-league',
                'EUROPA CONFERENCE LEAGUE GRP. G': 'uefa-europa-conference-league',
                'EUROPA CONFERENCE LEAGUE GRP. H': 'uefa-europa-conference-league','NORWAY - ELITESERIEN':'norway/eliteserien',
               'TÜRKIYE - SUPER LIG':'turkey/super-lig','GERMANY - 2. BUNDESLIGA':'germany/bundesliga-2',
               'SWEDEN - ALLSVENSKAN': 'sweden/allsvenskan', 'ENGLAND - LEAGUE ONE':'english/league-1',
               'ENGLAND - LEAGUE TWO':'english/league-2',
               'AUSTRALIA - A-LEAGUE MEN': 'world/australia/a-league'}

def get_all_odds_links(ltp):
    odds_links = {}
    for f_league, url_league in xg_leagues_o.items():
        if f_league not in ltp:
            continue
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.implicitly_wait(3)
        url = 'https://www.oddschecker.com/football/' + url_league
        driver.get(url)
        divs = driver.find_elements(By.XPATH, '//a[@class="beta-callout full-height-link whole-row-link"]')
        mt = []
        for div in divs:
            odds_url = div.get_attribute('href')
            mt.append(odds_url)
        odds_links[f_league] = mt[:12]
    return odds_links

In [4]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [5]:
def connect_links(data, all_odds):
    for l in data.keys():
        fotmob_names = data[l].keys()
        odds_names = {v: k for k, v in zip(all_odds[l],[str(i).split('/')[-2] for i in all_odds[l]])}
        for game in fotmob_names:
            match = getname(game, odds_names.keys())
            try:
                link = odds_names[match]
            except:
                link = ''
            data[l][game] = data[l][game] + [link]
    scraper_data = collections.defaultdict(list)
    for l_name, data in data.items():
        for game_name, l1_t_l2 in data.items():
            scraper_data[str(l1_t_l2[1])].append([game_name, l_name, l1_t_l2[0], l1_t_l2[2]])
    r = {}
    test = scraper_data.copy()
    for date, data in test.copy().items():
        if len(data) > 5:
            x = [data[i:i + 4] for i in range(0, len(data), 4)]
            d = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
            groups = len(x)
            times = [str(d + timedelta(seconds = i * 15)) for i in range(-groups//2+1,groups//2+1, 1)]
            new = dict(zip(times, x))
            del test[date]
            r = {**r, **new}
        else:
            r[date] = data
    scraper_data = dict(sorted(r.items(), key=lambda x: x[0]))
    return scraper_data